# Импорт библиотек

In [1]:
from utils import load_emoji_list, create_emoji_dict
import pickle

# Создание словаря смайликов

Этот ноутбук предназначен для создания словарей смайликов:
- `ключами` словаря служат сами смайлики;
- `значения` словаря представляют собой кортежи из среднего цвета смайлика в RGB-формате (numpy-массив) и доли занимаемой смайликом площади в квадратной ячейке (float от 0 до 1).

> Доля занимаемой площади нужна только для динамического окрашивания фона для каждого смайлика. Такая функция представлена в ноутбуке `dynamic_background.ipynb`.

Для определения среднего цвета каждый смайлик отрисовывается в квадрате NxN пикселей, затем по всем пикселям получившейся картинки вычисляется среднее значение по каналам R, G и B:
- за фон такой картинки отвечает параметр `background_color` (кортеж из трех значений RGB или None, если требуется прозрачный фон);
- за длину стороны квадрата N отвечает параметр `emoji_resolution`.

Пример словаря:
```python
{
    '😀': (array([222.66, 178.25,  68.99]), 0.78),
    '🐊': (array([ 93.66, 145.22,  61.83]), 0.33),
    '☀️': (array([249.23, 169.30,  50.14]), 0.36),
    '🍉': (array([193.92,  97.51,  84.73]), 0.39),
    '🎲': (array([217.24,  82.27, 101.94]), 0.55),
    '🇷🇺': (array([147.95, 106.95, 139.19]), 0.71)
}
```

In [2]:
# Название нового словаря смайликов (для сохранения)
emoji_dict_name = 'moon_black'
# Источник смайликов (файл из папки emoji_sources/)
emoji_list_name= 'moon'

In [3]:
%%time

# Создание словаря
emoji_dict = create_emoji_dict(emojis=load_emoji_list(emoji_list_name), 
                               emoji_resolution=100,
                               background_color=(0, 0, 0))

# Сохранение словаря
with open(f'emoji_dicts/{emoji_dict_name}.pkl', 'wb') as file:
    pickle.dump(emoji_dict, file)

# Вывод результатов
print(f'Словарь {emoji_dict_name} создан и сохранен:')
print(''.join(s for s in list(emoji_dict.keys())[:10]) + '...\n')

Построение словаря:  100.0% (8/8)
Словарь moon_black создан и сохранен:
🌕🌖🌗🌘🌑🌒🌓🌔...

CPU times: user 770 ms, sys: 68.7 ms, total: 838 ms
Wall time: 3.33 s
